计算Road的评分

In [2]:
import pandas as pd
import numpy as np

# 读取数据
data = pd.read_csv('data0.csv')

# 计算每条线路的平均运输量、最高运输量和运输量的方差
grouped_data = data.groupby(['place1', 'place2']).agg({'num': ['mean', 'max', 'var']})
grouped_data.columns = ['avg_num', 'max_num', 'var_num']
grouped_data.reset_index(inplace=True)

# 计算熵权法权重
def entropy_weight(data):
    p = data / data.sum()
    entropy = -(p * np.log(p)).sum()
    return (1 - entropy) / (1 - np.log(len(data)))

weights = grouped_data[['avg_num', 'max_num', 'var_num']].apply(entropy_weight)
print('特征权重:', weights)

# 数据标准化
normalized_data = grouped_data[['avg_num', 'max_num', 'var_num']] / grouped_data[['avg_num', 'max_num', 'var_num']].max()

# 加权标准化
weighted_data = normalized_data * weights

# 计算正负理想解
positive_ideal = weighted_data.max()
negative_ideal = weighted_data.min()

# 计算各方案到正负理想解的距离
distance_positive = np.sqrt(((weighted_data - positive_ideal) ** 2).sum(axis=1))
distance_negative = np.sqrt(((weighted_data - negative_ideal) ** 2).sum(axis=1))

# 计算相对接近度
relative_closeness = distance_negative / (distance_positive + distance_negative)

# 将相对接近度添加到原始数据表
grouped_data['closeness'] = relative_closeness

# 按相对接近度降序排列
sorted_data = grouped_data.sort_values('closeness', ascending=False)

# 保存结果到CSV文件
sorted_data.to_csv('Road_topsis_results.csv', index=False)

print('排序结果已保存到 Road_topsis_results.csv 文件。')


特征权重: avg_num    0.686574
max_num    0.725146
var_num    0.422346
dtype: float64
排序结果已保存到 Road_topsis_results.csv 文件。


d:\Applications\Development\anaconda\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


计算Node的评分

In [1]:
import pandas as pd
import numpy as np

# 读取数据
data = pd.read_csv('data0.csv')

# 计算每个结点的平均收货量、最高收货量、平均发货量、最高发货量
received = data.groupby('place2').agg({'num': ['mean', 'max']})
received.columns = ['avg_received', 'max_received']
received.reset_index(inplace=True)

sent = data.groupby('place1').agg({'num': ['mean', 'max']})
sent.columns = ['avg_sent', 'max_sent']
sent.reset_index(inplace=True)

# 计算与结点相关的线路数
routes_place1 = data.groupby('place1').agg({'place2': 'count'}).reset_index().rename(columns={'place1': 'place', 'place2': 'routes'})
routes_place2 = data.groupby('place2').agg({'place1': 'count'}).reset_index().rename(columns={'place2': 'place', 'place1': 'routes'})

# 合并统计信息
routes = pd.concat([routes_place1, routes_place2]).groupby('place').sum().reset_index()
node_data = pd.merge(pd.merge(received, sent, left_on='place2', right_on='place1'), routes, left_on='place1', right_on='place').drop(columns=['place1', 'place2'])

# 计算熵权法权重
def entropy_weight(data):
    p = data / data.sum()
    entropy = -(p * np.log(p)).sum()
    return (1 - entropy) / (1 - np.log(len(data)))

weights = node_data[['avg_received', 'max_received', 'avg_sent', 'max_sent', 'routes']].apply(entropy_weight)
print('特征权重:', weights)

# 数据标准化
normalized_data = node_data[['avg_received', 'max_received', 'avg_sent', 'max_sent', 'routes']] / node_data[['avg_received', 'max_received', 'avg_sent', 'max_sent', 'routes']].max()

# 加权标准化
weighted_data = normalized_data * weights

# 计算正负理想解
positive_ideal = weighted_data.max()
negative_ideal = weighted_data.min()

# 计算各方案到正负理想解的距离
distance_positive = np.sqrt(((weighted_data - positive_ideal) ** 2).sum(axis=1))
distance_negative = np.sqrt(((weighted_data - negative_ideal) ** 2).sum(axis=1))

# 计算相对接近度
relative_closeness = distance_negative / (distance_positive + distance_negative)

# 将相对接近度添加到原始数据表
node_data['closeness'] = relative_closeness

# 按相对接近度降序排列
sorted_data = node_data.sort_values('closeness', ascending=False)

# 保存结果到CSV文件
sorted_data.to_csv('Node_topsis_results.csv', index=False)

print('排序结果已保存到 Node_topsis_results.csv 文件。')


特征权重: avg_received    0.459899
max_received    0.443457
avg_sent        0.736051
max_sent        0.749402
routes          0.807868
dtype: float64
排序结果已保存到 Node_topsis_results.csv 文件。
